In [63]:
import os
import csv
import sys
import re
import pandas as pd

import heapq
from collections import defaultdict
from operator import itemgetter

from surprise import Dataset, Reader, SVD, accuracy, KNNBasic, KNNBaseline, KNNWithMeans
from surprise.model_selection import train_test_split
from collections import defaultdict
import numpy as np

In [10]:
class MovieLens:

    movieID_to_name = {}
    name_to_movieID = {}
    ratingsPath = 'C:\\Users\\joseh\\Anaconda Projects\\Personal studies\\RecSys Course\\ml-latest-small\\ratings ut.csv'
    moviesPath = 'C:\\Users\\joseh\\Anaconda Projects\\Personal studies\\RecSys Course\\ml-latest-small\\movies.csv'
    
    def loadMovieLensLatestSmall(self):

        # Look for files relative to the directory we are running from
        os.chdir(os.path.dirname(sys.argv[0]))

        ratingsDataset = 0
        self.movieID_to_name = {}
        self.name_to_movieID = {}

        reader = Reader(line_format='user item rating timestamp', sep=',', skip_lines=1)

        ratingsDataset = Dataset.load_from_file(self.ratingsPath, reader=reader)

        with open(self.moviesPath, newline='', encoding='ISO-8859-1') as csvfile:
                movieReader = csv.reader(csvfile)
                next(movieReader)  #Skip header line
                for row in movieReader:
                    movieID = int(row[0])
                    movieName = row[1]
                    self.movieID_to_name[movieID] = movieName
                    self.name_to_movieID[movieName] = movieID

        return ratingsDataset
    
    def getMovieName(self, movieID):
        if movieID in self.movieID_to_name:
            return self.movieID_to_name[movieID]
        else:
            return ""
        
    def getMovieID(self, movieName):
        if movieName in self.name_to_movieID:
            return self.name_to_movieID[movieName]
        else:
            return 0

In [11]:
ml = MovieLens()
data = ml.loadMovieLensLatestSmall()

In [38]:
trainSet = data.build_full_trainset()

In [49]:
testSubject = '672'

In [65]:
# User-based KNN
UserKNN = KNNWithMeans(sim_options = {'name': 'msd', 'user_based': True})

# Item-based KNN
ItemKNN = KNNBasic(sim_options = {'name': 'msd', 'user_based': False})

In [66]:
UserKNN.fit(trainSet)
            
print("Computing recommendations...")

fill = trainSet.global_mean
anti_testset = []
u = trainSet.to_inner_uid(str(testSubject))
user_items = set([j for (j, _) in trainSet.ur[u]])
anti_testset += [(trainSet.to_raw_uid(u), trainSet.to_raw_iid(i), fill) for
                         i in trainSet.all_items() if
                         i not in user_items]

predictions = UserKNN.test(anti_testset)

recommendations = []

print ("\nWe recommend:")
for userID, movieID, actualRating, estimatedRating, _ in predictions:
    intMovieID = int(movieID)
    recommendations.append((intMovieID, estimatedRating))

recommendations.sort(key=lambda x: x[1], reverse=True)

for ratings in recommendations[:10]:
    print(ml.getMovieName(ratings[0]), ratings[1])

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing recommendations...

We recommend:
Step Into Liquid (2002) 5
Walking and Talking (1996) 5
Daytrippers, The (1996) 5
Taste of Cherry (Ta'm e guilass) (1997) 5
Character (Karakter) (1997) 5
Ipcress File, The (1965) 5
Rosetta (1999) 5
Yojimbo (1961) 5
War Zone, The (1999) 5
Mifune's Last Song (Mifunes sidste sang) (1999) 5


In [54]:
user_items

{106, 143, 278, 412, 417, 480, 1527, 1753, 1796}